## Graph Based Recommendations With Neo4j, NLP, and Python Data Science Tools

In [ ]:
!pip install py2neo
!pip 
!pip install summa
!pip install python-igraph

## Import data
- show the domain
- show the CSV heads
- 

In [1]:
from py2neo import Graph
graph = Graph()

### Import Groups and Topics
![](http://guides.neo4j.com/bostonmeetup/img/group_has_topic.png)

In [ ]:
graph.run("CREATE CONSTRAINT ON (g:Group) ASSERT g.id IS UNIQUE;")

In [ ]:
graph.run("CREATE CONSTRAINT ON (t:Topic) ASSERT t.id IS UNIQUE;")

In [ ]:
graph.run("CREATE INDEX ON :Group(name)")

In [ ]:
graph.run("CREATE INDEX ON :Topic(name)")

In [ ]:
graph.run('''
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/groups.csv"
AS row
MERGE (group:Group { id:row.id })
ON CREATE SET
  group.name = row.name,
  group.urlname = row.urlname,
  group.rating = toInt(row.rating),
  group.created = toInt(row.created)
''')

In [ ]:
graph.run('''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/groups_topics.csv"  AS row
MERGE (topic:Topic {id: row.id})
ON CREATE SET topic.name = row.name, topic.urlkey = row.urlkey
''')

In [ ]:
graph.run('''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/groups_topics.csv"  AS row
MATCH (topic:Topic {id: row.id})
MATCH (group:Group {id: row.groupId})
MERGE (group)-[:HAS_TOPIC]->(topic)
''')

### Find similar groups to Graph Database Boston
By looking at topics, can we find groups that have similar topics to Graph Database Boston?

In [ ]:
result = graph.run('''
MATCH (group:Group)-[:HAS_TOPIC]->(topic)<-[:HAS_TOPIC]-(otherGroup)
WHERE group.name CONTAINS "Graph Database"
RETURN otherGroup.name, COUNT(topic) AS topicsInCommon,
       COLLECT(topic.name) AS topics
ORDER BY topicsInCommon DESC, otherGroup.name
LIMIT 10
''')

for row in result:
    print(row)

## Topic Similarity

Using community detection to find similar topics

![](https://camo.githubusercontent.com/0054c52996fdd9bb82456406cb867bdb3985d14c/687474703a2f2f7777772e6c796f6e776a2e636f6d2f7075626c69632f696d672f636f6d6d756e6974792d312e706e67)

In [2]:
from igraph import Graph as IGraph

Find all pairs of topics and find the number of common groups that share each pair of topics. We'll use this as weight to build a "virtual graph" of the form`(Topic)-[:OCCURS_WITH {weight}]-(Topic)`

In [3]:

query = """
MATCH (topic:Topic)<-[:HAS_TOPIC]-()-[:HAS_TOPIC]->(other:Topic)
WHERE ID(topic) < ID(other)
RETURN topic.name, other.name, COUNT(*) AS weight
ORDER BY weight DESC
LIMIT 10
"""

result = graph.run(query)
for row in result:
    print(row)


(u'topic.name': u'Software Development', u'other.name': u'Computer programming', u'weight': 101)
(u'topic.name': u'Web Development', u'other.name': u'Software Development', u'weight': 88)
(u'topic.name': u'Open Source', u'other.name': u'Software Development', u'weight': 77)
(u'topic.name': u'New Technology', u'other.name': u'Software Development', u'weight': 72)
(u'topic.name': u'Big Data', u'other.name': u'Data Analytics', u'weight': 68)
(u'topic.name': u'Web Technology', u'other.name': u'Software Development', u'weight': 65)
(u'topic.name': u'Web Development', u'other.name': u'Computer programming', u'weight': 60)
(u'topic.name': u'Big Data', u'other.name': u'Big Data Analytics', u'weight': 59)
(u'topic.name': u'Open Source', u'other.name': u'Computer programming', u'weight': 54)
(u'topic.name': u'Big Data', u'other.name': u'Data Science', u'weight': 51)


Now let's run this query again and build an igraph instance from the results:

In [4]:
query = """
MATCH (topic:Topic)<-[:HAS_TOPIC]-()-[:HAS_TOPIC]->(other:Topic)
WHERE ID(topic) < ID(other)
RETURN topic.name, other.name, COUNT(*) AS weight
"""

ig = IGraph.TupleList(graph.run(query), weights=True)
ig

Now we'll run the Walktrap community detection algorithm to find clusters / communities:

In [5]:
clusters = IGraph.community_walktrap(ig, weights="weight")
clusters = clusters.as_clustering()
len(clusters)

39

Let's inspect the results:

In [6]:

nodes = [node["name"] for node in ig.vs]
nodes = [{"id": x, "label": x} for x in nodes]
nodes[:5]

for node in nodes:
    idx = ig.vs.find(name=node["id"]).index
    node["group"] = clusters.membership[idx]
    
nodes[:5]

[{'group': 0,
  'id': u'Software QA and Testing',
  'label': u'Software QA and Testing'},
 {'group': 0, 'id': u'Test Automation', 'label': u'Test Automation'},
 {'group': 1, 'id': u'Game Development', 'label': u'Game Development'},
 {'group': 2, 'id': u'Games', 'label': u'Games'},
 {'group': 3,
  'id': u'MicroStrategy Best Practices',
  'label': u'MicroStrategy Best Practices'}]

Now we'll write the results back to Neo4j, extended our graph model:
![](http://guides.neo4j.com/bostonmeetup/img/cluster_datamodel.png)

In [7]:
query = """
UNWIND {params} AS p 
MATCH (t:Topic {name: p.id}) 
MERGE (cluster:Cluster {name: p.group})
MERGE (t)-[:IN_CLUSTER]->(cluster)
"""

graph.run(query, params = nodes)

We can see which clusters the Python related topics end up being in:
![](http://guides.neo4j.com/bostonmeetup/img/python_cluster.png)

In [ ]:
graph.run('''
MATCH (cluster:Cluster)<-[inCluster:IN_CLUSTER]-(topic)
WHERE topic.name CONTAINS "Python"
RETURN *
''')

## My Similar Groups

We need to add Member data in order to build more relevant
recommendations:
![](http://guides.neo4j.com/bostonmeetup/img/group_has_topic_member_of.png)

In [ ]:
graph.run('''
CREATE CONSTRAINT ON (m:Member)
ASSERT m.id IS UNIQUE''')

In [ ]:
graph.run('''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/members.csv" AS row
WITH DISTINCT row.id AS id, row.name AS name
MERGE (member:Member {id: id})
ON CREATE SET member.name = name
''')

In [ ]:
graph.run('''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/members.csv" AS row
WITH row WHERE NOT row.joined is null
MATCH (member:Member {id: row.id})
MATCH (group:Group {id: row.groupId})
MERGE (member)-[membership:MEMBER_OF]->(group)
ON CREATE SET membership.joined=toInt(row.joined);
''')

In [ ]:
graph.run('''
MATCH (member:Member)-[membership:MEMBER_OF]->(group)
RETURN member, group, membership
LIMIT 10
''')

![](http://guides.neo4j.com/bostonmeetup/img/group_members.png)

In [ ]:
graph.run("CREATE INDEX ON :Member(name)")

### Find my similar groups

In [ ]:
results = graph.run('''MATCH (member:Member {name: "Will Lyon"})-[:MEMBER_OF]->()-[:HAS_TOPIC]->()<-[:HAS_TOPIC]-(otherGroup:Group)
WHERE NOT (member)-[:MEMBER_OF]->(otherGroup)
RETURN otherGroup.name,
       COUNT(*) AS topicsInCommon
ORDER BY topicsInCommon DESC
LIMIT 10''')

for row in results:
    print(row)

## Events
![](http://guides.neo4j.com/bostonmeetup/img/event_datamodel.png)

In [ ]:
graph.run("CREATE CONSTRAINT ON (e:Event) ASSERT e.id IS UNIQUE")

In [ ]:
graph.run("CREATE INDEX ON :Event(time)")

In [ ]:
graph.run('''USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/events.csv" AS row
MERGE (event:Event {id: row.id})
ON CREATE SET event.name = row.name,
              event.description = row.description,
              event.time = toInt(row.time),
              event.utcOffset = toInt(row.utc_offset)
''')
              

In [ ]:
graph.run('''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/johnymontana/harvard-bar/master/data/events.csv" AS row

WITH distinct row.group_id as groupId, row.id as eventId
MATCH (group:Group {id: groupId})
MATCH (event:Event {id: eventId})
MERGE (group)-[:HOSTED_EVENT]->(event)
''')

In [ ]:
graph.run('''
MATCH (group:Group)-[hosted:HOSTED_EVENT]->(event)
WHERE group.name CONTAINS "Graph Database" AND event.time < timestamp()
RETURN event, group, hosted
ORDER BY event.time DESC
LIMIT 10
''')

### Adding Member RSVPs
TODO: add datamodel image

In [ ]:
graph.run('''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "file:///rsvps.csv" AS row
WITH row WHERE row.response = "yes"

MATCH (member:Member {id: row.member_id})
MATCH (event:Event {id: row.event_id})
MERGE (member)-[rsvp:RSVPD {id: row.rsvp_id}]->(event)
ON CREATE SET rsvp.created = toint(row.created),
              rsvp.lastModified = toint(row.mtime),
              rsvp.guests = toint(row.guests)
''')

![](![](http://guides.neo4j.com/bostonmeetup/img/graph_database_events.png)

### Extracting keywords from event descriptions
Note that we have topics for groups, but not for Events. We can use some NLP techniques to extract keywords from event descriptions and extend our datamodel to take those keywords into account in our recommendation queries.

![](http://guides.neo4j.com/bostonmeetup/img/keyword_datamodel.png)

In [8]:
# Helper function for stripping HTML
from HTMLParser import HTMLParser
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)
    
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [9]:
from summa import keywords

In [10]:
graph.run("CREATE CONSTRAINT ON (k:Keyword) ASSERT k.name IS UNIQUE")

In [11]:
def addKeywords():
    result = graph.run('''
        MATCH (e:Event)<-[:HOSTED_EVENT]-(g:Group {name: "Graph Database Boston"})
        RETURN e.id AS e_id, e.description AS desc
        ''')
    for row in result:
        text = strip_tags(row['desc'])
        kws = keywords.keywords(text)
        if kws:
            k = [str(k) for k in kws.split('\n')]
            
            p = {
                'kws': k,
                'e_id': str(row['e_id'])
            }
            print(p)
            
            query = '''
            WITH {kws} AS kws
                MATCH (e:Event) WHERE e.id = {e_id}
                UNWIND kws AS kw
                MERGE (k:Keyword {name: kw})
                MERGE (e)-[:HAS_TAG]->(k)
            '''
            
            graph.run(query, parameters = p)


In [12]:
addKeywords()

{'kws': ['data', 'recommendation', 'recommendations', 'william', 'build', 'building', 'neo', 'language', 'graph', 'graphs', 'science', 'like', 'documentation', 'feel', 'networking', 'talk', 'pizza', 'software developer', 'develop', 'developers'], 'e_id': '233998812'}
{'kws': ['meetup'], 'e_id': '232416626'}
{'kws': ['successful', 'success team', 'customer', 'enterprise customers', 'management', 'account', 'dave', 'process', 'gordon', 'leads', 'technical'], 'e_id': '231623217'}
{'kws': ['neo', 'data', 'pass', 'hadoop', 'strata', 'include', 'ticket included', 'uggdbb'], 'e_id': '198290172'}
{'kws': ['data', 'dave', 'neo', 'changes', 'technology', 'new', 'providing', 'provided', 'area', 'release'], 'e_id': '218826807'}
{'kws': ['meetup', 'code', 'price', 'ticket', 'sessions'], 'e_id': '219791058'}
{'kws': ['webinar', 'http', 'session', 'pm', 'data'], 'e_id': '221232206'}
{'kws': ['data', 'visualization', 'visualizing', 'visually', 'graph', 'cambridge', 'solutions', 'solution', 'query', 'j

In [ ]:
type(kws)

In [ ]:
s = kws.split('\n')

In [ ]:
s